In [40]:
import nltk, re, string
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from unidecode import unidecode
from nltk.sentiment import SentimentIntensityAnalyzer

In [41]:
# Spécifier le chemin du fichier PDF à partir duquel on va extraire le texte.
txt_path = r'C:\Users\smplon\Desktop\mandat.txt'

# Ouvrir le fichier texte en mode lecture avec 'with open'
with open(txt_path, 'r', encoding='utf-8') as txt_file:
    # Lire le contenu du fichier texte
    texte = txt_file.read()

In [42]:
texte

"\ufeffAvis : Contre\nJustification : Bien qu’il ait accompli beaucoup durant ses 3 précédents mandats, je pense qu’il est temps que le président Ouattara laisse la place à la nouveauté. De plus, son alliance avec la France qui au début semblait l'avantage face à ses concurrents (Gbagbo notamment) présente aujourd’hui un défaut voire une faiblesse. En effet, la Côte d’Ivoire n’échappe pas au vent de coup d’état qui semble souffler dans toute l’Afrique francophone. La jeunesse (ivoirienne dans ce cas) en a assez de se sentir paterner par la puissance française. Nous avons pris notre indépendance il y a plus de 60 ans, il est temps de l’assumer désormais.\n\nAvis : Contre\nJustification : Je suis fière de l'évolution de mon pays la côte d’ivoire, fière du leadership de mon président mais pour moi notre notre pays gagnerait à avoir la jeunesse au commande. Cette jeunesse dirigeante pourrait alors se tourner vers nos anciens présidents et avoir plus d'expérience. \n\nAvis : Contre \nJustif

In [43]:
# Utilisation d'une expression régulière pour extraire les avis
avis_pattern = r"Avis : ([\s\S]*?)(?=(?:Avis :|$))"

avis_extraits = re.findall(avis_pattern, texte)

# Supprimer les espaces vides et les sauts de ligne en début et en fin de chaque avis
avis_propres = [avis.strip() for avis in avis_extraits]

# # Affichez les avis extraits
for i, avis in enumerate(avis_propres, 1):
    print(avis)

Contre
Justification : Bien qu’il ait accompli beaucoup durant ses 3 précédents mandats, je pense qu’il est temps que le président Ouattara laisse la place à la nouveauté. De plus, son alliance avec la France qui au début semblait l'avantage face à ses concurrents (Gbagbo notamment) présente aujourd’hui un défaut voire une faiblesse. En effet, la Côte d’Ivoire n’échappe pas au vent de coup d’état qui semble souffler dans toute l’Afrique francophone. La jeunesse (ivoirienne dans ce cas) en a assez de se sentir paterner par la puissance française. Nous avons pris notre indépendance il y a plus de 60 ans, il est temps de l’assumer désormais.
Contre
Justification : Je suis fière de l'évolution de mon pays la côte d’ivoire, fière du leadership de mon président mais pour moi notre notre pays gagnerait à avoir la jeunesse au commande. Cette jeunesse dirigeante pourrait alors se tourner vers nos anciens présidents et avoir plus d'expérience.
Contre 
Justification : Je suis contre le quatrième 

In [49]:
# Mettez le texte nettoyé en minuscules
avis_propres = [avis.lower() for avis in avis_propres]

# Supprimez la ponctuation, les accents et effectuez la lemmatisation
avis_propres_traite = []

for avis in avis_propres:

    # Supprimer la ponctuation et les caractères spéciaux
    avis = re.sub(f"[{string.punctuation}]", "", avis)
    
    # Supprimer les accents
    avis = unidecode(avis)
    
    # Tokenization en mots
    mots = nltk.word_tokenize(avis)
    
    # Supprimer les mots vides (stop words)
    mots = [mot for mot in mots if mot not in stopwords.words('french')]
    
    # Effectuer la lemmatisation
    avis_traite = " ".join([word.lemmatize() for word in TextBlob(" ".join(mots)).words])
    
    # Supprimer les lettres non prises en charge par la lemmatisation
    avis_traite = re.sub(r'\b\w{1,2}\b', '', avis_traite)
    
    avis_propres_traite.append(avis_traite)

# La liste avis_propres_traite contient maintenant les avis traités avec la lemmatisation
avis_propres_traite

["contre justification bien ' accompli beaucoup durant  precedent mandats pense ' temp president ouattara laisse place  nouveaute plus alliance france debut semblait lavantage face  concurrents gbagbo notamment presente aujourd'hui defaut voire faiblesse effet cote 'ivoire 'echappe vent coup 'etat semble souffler toute 'afrique francophone jeunesse ivoirienne   assez sentir paterner puissance francaise pris independance  plus   temp 'assumer desormais",
 "contre justification fiere levolution pay cote 'ivoire fiere leadership president pay gagnerait  avoir jeunesse commande cette jeunesse dirigeante pourrait alors tourner vers anciens president avoir plus dexperience",
 'contre justification contre quatrieme mandat president ouattara simple fait quil exerce lillegalite 3eme mandat inconstitutionnelle meme 4eme battre quand opposant cause refaire meme erreurs lesquelles sest battu decevant progres oui detriment liberte peuple',
 "contre justification bien rhdp constate ' temp laisser pl

In [53]:
# Analyse de sentiment à l'aide de NLTK
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialisation de l'analyseur de sentiment NLTK
sia = SentimentIntensityAnalyzer()

# Liste pour stocker les résultats de l'analyse de sentiment de chaque avis
resultats_sentiment_nltk = []

# Analyse de sentiment pour chaque avis
for avis in avis_propres_traite:
    sentiment = sia.polarity_scores(avis)
    resultats_sentiment_nltk.append(sentiment)

# Mots-clés associés aux avis positifs
mots_cles_positifs = ["positif", "soutien", "favorable", "content", "satisfait", "terminer", "rester"]

# Mots-clés associés aux avis négatifs
mots_cles_negatifs = ["dictature", "respecte", "contre", "désaccord", "problème", "opposition", "déception", "insatisfait", "temp", "laisse", "nouvelle", "su" ]

# Liste pour stocker les polarités des avis basées sur les mots-clés
polarites_avis_mots_cles = []

# Analyse de chaque avis basée sur les mots-clés
for avis in avis_propres_traite:
    avis = avis.lower()  # Convertir en minuscules pour la correspondance insensible à la casse
    
    # Compter le nombre de mots-clés positifs et négatifs
    count_positifs = sum(1 for mot in mots_cles_positifs if mot in avis)
    count_negatifs = sum(1 for mot in mots_cles_negatifs if mot in avis)
    
    # Attribuer une polarité basée sur les mots-clés
    if count_positifs > count_negatifs:
        polarite = "positif"
    elif count_negatifs > count_positifs:
        polarite = "négatif"
    else:
        polarite = "neutre"
    
    polarites_avis_mots_cles.append(polarite)

# Combiner les polarités basées sur le sentiment et les mots-clés
polarites_avis_combinees = []

for i in range(len(resultats_sentiment_nltk)):
    polarite_sentiment = resultats_sentiment_nltk[i]
    polarite_mots_cles = polarites_avis_mots_cles[i]
    
    # Logique de combinaison des polarités (à personnaliser)
    if polarite_sentiment['compound'] >= 0.05:
        polarite_combinee = "positif"
    else:
        polarite_combinee = polarite_mots_cles
    
    polarites_avis_combinees.append(polarite_combinee)

# Afficher les polarités attribuées aux avis
for i, polarite in enumerate(polarites_avis_combinees, 1):
    print(f"Avis {i}: {polarite}")


Avis 1: négatif
Avis 2: négatif
Avis 3: négatif
Avis 4: négatif
Avis 5: positif
Avis 6: neutre
Avis 7: positif
Avis 8: négatif
Avis 9: négatif
Avis 10: négatif
Avis 11: négatif
Avis 12: négatif
Avis 13: positif
Avis 14: négatif
Avis 15: négatif
Avis 16: négatif


In [51]:
# Compteur pour chaque polarité
compteur_polarites = {
    "positif": 0,
    "négatif": 0,
    "neutre": 0
}

# Parcourir les polarités attribuées aux avis et les compter
for polarite in polarites_avis_combinees:
    if polarite == "positif":
        compteur_polarites["positif"] += 1
    elif polarite == "négatif":
        compteur_polarites["négatif"] += 1
    elif polarite == "neutre":
        compteur_polarites["neutre"] += 1

# Afficher le nombre total de chaque polarité
print("Nombre d'avis positifs :", compteur_polarites["positif"])
print("Nombre d'avis négatifs :", compteur_polarites["négatif"])
print("Nombre d'avis neutres :", compteur_polarites["neutre"])


Nombre d'avis positifs : 3
Nombre d'avis négatifs : 12
Nombre d'avis neutres : 1
